# Práctica 2: Aprendizaje y selección de modelos de clasificación

## Minería de Datos

### Curso académico 2022-2023

### Alumnos:

* Pablo Rodríguez Royo
* Roberto Ibáñez Gómez


# 0. Preliminares

Antes de empezar importamos todas las librerias necesarias.

In [126]:
# Standard
import os
from pathlib import Path

# Third party
from sklearn.metrics import check_scoring, make_scorer, accuracy_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from functools import partial

import pandas as pd
import time

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
random_state = 27912

Ahora definimos una función que nos mostrará una tabla ordenada con los mejores hiperparámetros, para nuestro modelo. Este mñetodo ha sido modificado para que podamos pasarle los hiperparámetros como un diccionario o una lista de diccionarios en el caso de Bagging donde hemos utilizado dos tipos de estimadores para el ensemble.

In [90]:
def optimize_params(pipeline, X, y, cv, scoring=None, refit=True, param_grid=None, **param_grid1):
    """Exhaustive search over specified parameter values for an estimator."""
    
    if (param_grid is None):
        grid_search_cv = GridSearchCV(pipeline,
                                  param_grid1,
                                  scoring=scoring,
                                  refit=refit,
                                  cv=cv,
                                  return_train_score=True).fit(X, y)
    else:
        grid_search_cv = GridSearchCV(pipeline,
                                param_grid,
                                scoring=scoring,
                                refit=refit,
                                cv=cv,
                                return_train_score=True).fit(X, y)


    cv_results = pd.DataFrame(grid_search_cv.cv_results_)

    # Drop the results for each validation split and sort by the refit metric
    labels = cv_results.filter(regex="split")
    by = cv_results.filter(regex="rank_test").columns[0]
    cv_results = cv_results.drop(labels, axis=1).sort_values(by)

    display(cv_results)

    return grid_search_cv

Ahora otra función más para validar nuestro conjunto de prueba, con determinados tipos de modelo dando una métrica en concreto. Este método ha sido modificado por nosotros ya que teniamos problemas con el check_scoring y las métricas recall y f1, puesto que teniamos que ajustar el parámetro `pos_label` al no haber codificado nuestra variable clase.

In [165]:
def evaluate_estimators(estimators, metrics, X, y):
    """Evaluate the estimators using the specified metrics."""
    results = pd.DataFrame(columns=metrics + ["Tiempo de inferencia"])
   
    for estimator in estimators:
        # Set the index of the results to the estimator class name, which may be may be a model or a pipeline
        name = (estimator[-1] if isinstance(estimator, Pipeline) else estimator).estimator[-1].__class__.__name__
        elapsed_time_total = 0
        start_time = time.time()
        y_pred = estimator.predict(X)
        final_time = time.time()
        elapsed_time = final_time - start_time
        elapsed_time_total += elapsed_time
        for metric in metrics:
            # Determine the scorer for evaluating the estimator
            if metric == "accuracy":
                results.loc[name, metric] = accuracy_score(y, y_pred)
            elif metric == "recall":
                results.loc[name, metric] = recall_score(y, y_pred, pos_label = 'T')
            elif metric == "f1":
                results.loc[name, metric] = f1_score(y, y_pred, pos_label = 'T')
        results.loc[name, "Tiempo de inferencia"] = elapsed_time_total

    return results

# 1. Carga de Datos

In [7]:
path = Path(".") / "data" / "csgo_round_snapshots.csv"
data = pd.read_csv(path, dtype={"round_winner": "category"})

Comprobamos que se ha cargado correctamente:

In [8]:
data.sample(5, random_state=random_state)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
11812,174.91,4.0,0.0,de_overpass,False,500.0,500.0,369.0,0.0,36200.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,CT
47872,34.94,7.0,13.0,de_mirage,False,396.0,499.0,400.0,499.0,22600.0,...,4.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,T
10223,169.89,6.0,3.0,de_nuke,False,500.0,500.0,396.0,0.0,17200.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,CT
93829,174.91,3.0,8.0,de_vertigo,False,500.0,500.0,0.0,395.0,28350.0,...,3.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,T
56994,175.00,11.0,4.0,de_train,False,500.0,500.0,300.0,200.0,1350.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T


Dividimos en variables predictoras y variable clase:

In [9]:
X = data.drop("round_winner", axis=1)
y = data["round_winner"]

Comprobamos que se ha separado correctamente.

In [10]:
X.sample(5, random_state=random_state)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
11812,174.91,4.0,0.0,de_overpass,False,500.0,500.0,369.0,0.0,36200.0,...,4.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
47872,34.94,7.0,13.0,de_mirage,False,396.0,499.0,400.0,499.0,22600.0,...,2.0,4.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0
10223,169.89,6.0,3.0,de_nuke,False,500.0,500.0,396.0,0.0,17200.0,...,2.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
93829,174.91,3.0,8.0,de_vertigo,False,500.0,500.0,0.0,395.0,28350.0,...,0.0,3.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
56994,175.00,11.0,4.0,de_train,False,500.0,500.0,300.0,200.0,1350.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y.sample(5, random_state=random_state)

11812    CT
47872     T
10223    CT
93829     T
56994     T
Name: round_winner, dtype: category
Categories (2, object): ['CT', 'T']

Dividimos el conjunto de datos en entrenamiento y prueba de forma estratificada. Esto, asegura que la distribución de clases en el conjunto de entrenamiento y en el conjunto de prueba sea lo más similar posible a la distribución de clases en el conjunto de datos completo.

In [12]:
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

In [13]:
X_train.sample(5, random_state=random_state)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
49096,28.73,11.0,7.0,de_vertigo,True,200.0,400.0,200.0,381.0,350.0,...,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
111328,66.84,2.0,0.0,de_nuke,False,100.0,246.0,100.0,367.0,1300.0,...,1.0,3.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
62440,114.94,0.0,2.0,de_overpass,False,500.0,500.0,84.0,475.0,11850.0,...,0.0,4.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0
31209,74.89,4.0,6.0,de_overpass,False,494.0,500.0,500.0,399.0,3050.0,...,3.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
58531,174.93,10.0,4.0,de_vertigo,False,500.0,500.0,197.0,0.0,24900.0,...,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_train.sample(5, random_state=random_state)

49096      T
111328     T
62440      T
31209      T
58531     CT
Name: round_winner, dtype: category
Categories (2, object): ['CT', 'T']

In [15]:
X_test.sample(5, random_state=random_state)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
23756,62.59,2.0,7.0,de_mirage,False,300.0,240.0,300.0,370.0,7600.0,...,0.0,2.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
25550,34.94,10.0,9.0,de_inferno,False,400.0,276.0,366.0,293.0,15050.0,...,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
106399,36.06,2.0,10.0,de_dust2,True,179.0,200.0,200.0,182.0,5750.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65103,175.00,2.0,2.0,de_train,False,500.0,500.0,100.0,373.0,9000.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114003,86.88,0.0,1.0,de_dust2,False,200.0,304.0,200.0,365.0,400.0,...,1.0,2.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0


In [16]:
y_test.sample(5, random_state=random_state)

23756     T
25550     T
106399    T
65103     T
114003    T
Name: round_winner, dtype: category
Categories (2, object): ['CT', 'T']

# 2. Preprocesamiento

Teniendo en cuenta que vamos a utilizar el mismo conjunto de datos de la práctica anterior, vamos a preprocesarlo con las técnicas ya vistas en la práctica 1, todo a través del pipeline avanzado. Esto nos facilitará el trabajo y nos ahorrara tiempo de ejecución.

In [17]:
from pprint import pprint
from sklearn.utils import all_estimators
from sklearn.tree import plot_tree
from functools import partial
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
import utils as ut
import joblib

In [18]:
from sklearn import set_config

set_config(transform_output="pandas")

In [19]:
def create_helmet(X):
    return X.assign(helmet=(X["t_helmets"] - X["ct_helmets"]))

La libreria joblib nos permite cargar nuestro pipeline del preprocesamiento sin tener que volver a escribir todo el código del preprocesamiento de la anterior práctica.

In [20]:
preprocesamiento = joblib.load("preprocesamiento.pkl")
preprocesamiento

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                ('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop', 'drop',
                                                  ['t_weapon_famas',
                                                   'ct_weapon_galilar',
                                                   't_weapon_usps',
                                                   'ct_weapon_glock',
                                                   't_weapon_fiveseven',
                                                   't_weapon_p2000',
                                                   'ct_weapon_tec9',
                                                   'ct_weapon_mac10',
                                                   '...
                                                   't_grenade_decoygrenade',
                                                   'ct_weapon_bizon',
                                                   'ct_weapon_g3sg1',
                                                   'ct_weapon_m249',
                                                   'ct_weapon_mac10', ...])],
                                   verbose_feature_names_out=False)),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['bomb_planted', 'map'])],
                                   verbose_feature_names_out=False))])

# 3. Modelos de clasificación supervisada

## 3.1 Vecinos más Cercanos

Este método se basa en un principio intuitivo: que puntos similares en un espacio de características tienden a pertenecer a la misma categoría. En k-NN, elegimos un número 'k' que representa la cantidad de vecinos más cercanos a considerar para la clasificación de un nuevo punto.

Cuando tenemos que clasificar un nuevo dato, lo que hace el algoritmo es buscar los 'k' puntos más próximos en nuestro conjunto de entrenamiento. La proximidad se mide usualmente mediante distancias, como la distancia Euclidia o la de Manhattan. La clasificación se determina entonces por un sistema de votación mayoritaria: asignamos al nuevo punto la clase que sea más frecuente entre estos 'k' vecinos más cercanos.

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [22]:
standard_scaler = StandardScaler()

In [23]:
n_neighbors = 5

k_neighbors_model = KNeighborsClassifier(n_neighbors)
vecinos = make_pipeline(preprocesamiento,standard_scaler, k_neighbors_model)
vecinos

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

## 3.2 Arboles de Decisión

Los árboles de decisión funcionan dividiendo el espacio de características en regiones más pequeñas y manejables (lo que nosostros conocemos como cortes de guillotina informalmente). Imaginemos el árbol de decisión como un conjunto de preguntas sucesivas, donde cada pregunta se basa en una característica y su respuesta nos lleva a la siguiente pregunta, hasta llegar a una decisión final.

El proceso comienza en la raíz del árbol, donde se elige la característica que mejor divide los datos según criterios como la ganancia de información, el índice de Gini o la reducción de la varianza. Esta división crea dos o más ramas, cada una representando una respuesta a la pregunta planteada. Este proceso se repite en cada rama, creando subárboles, hasta que se cumplen ciertas condiciones, como que todos los datos en un nodo pertenezcan a la misma clase o que se alcance una profundidad máxima predefinida para el árbol.

In [24]:
decision_tree = DecisionTreeClassifier(random_state=random_state)

pipeline_arbol = make_pipeline(preprocesamiento, decision_tree)
pipeline_arbol

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_m249',
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=27912))])

## 3.3 Adaptative Boosting

El modelo consiste en combinar **múltiples modelos simples**, conocidos también como 1R o decision stumps, para formar un clasificador más fuerte y preciso.

El proceso de AdaBoost comienza con la asignación de un peso igual a cada muestra en el conjunto de datos. Luego, se entrena un stump, como un árbol de decisión simple, en estos datos. Después de entrenar este clasificador, AdaBoost evalúa su rendimiento y ajusta los pesos de las instancias. Específicamente, aumenta los pesos de las instancias que fueron clasificadas incorrectamente, de modo que el próximo clasificador se enfoque más en estas instancias.

Este proceso se repite varias veces, entrenando nuevos stumps en cada iteración. **Cada clasificador se entrena teniendo en cuenta el rendimiento de los clasificadores anteriores**. Al final, AdaBoost combina todos estos stumps en un solo modelo. Cada clasificador débil tiene una influencia en la decisión final basada en su precisión; los más precisos tienen más peso.

In [102]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

adaptative = make_pipeline(preprocesamiento, adaboost_model)
adaptative

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_m249',
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('adaboostclassifier', AdaBoostClassifier(random_state=27912))])

## 3.4 Bootstrap Aggregating

El Bagging es un algoritmo que busca mejorar la precisión de los modelos de aprendizaje al **combinar varios clasificadores**, cada uno **entrenado en una muestra aleatoria con reemplazo** (`bootstrap`) del conjunto de datos original. Todas las muestras son del mismo tamaño $D$ conjunto de datos original. La diversidad entre los modelos ayuda a reducir la varianza y el sobreajuste, mejorando así la generalización a nuevos datos. El resultado final se obtiene mediante **votación por mayoría para clasificación** (`aggregating`) o promedio para regresión. Esto se hace obteniendo la moda de {$y_1$...$y_t$} siendo estas las salidas obtenidas de los modelos. Bagging suele mejorar la precisión y es efectivo en conjuntos de datos ruidosos, tiene desventajas como el alto costo computacional y la menor interpretabilidad en comparación con otros métodos, como los árboles de decisión. El equilibrio entre sesgo y varianza es crucial para lograr un modelo generalizable. 

In [93]:
bagging_model = BaggingClassifier(random_state=random_state)

bagging = make_pipeline(preprocesamiento,standard_scaler, bagging_model)
bagging

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('standardscaler', StandardScaler()),
                ('baggingclassifier', BaggingClassifier(random_state=27912))])

## 3.5 Random Forest

`Random Forest` es una técnica de aprendizaje automático supervisado, el cual es un ensemble formado por árboles de decisión. Estos árboles se entrenan escogiendo muestras aleatorias de nuestro conjunto de datos con reemplazo (bootstrap) y escogiendo para cada split de un árbol del esemble características aleatorias, esto se hace para reducir la varianza de cada estimador del esemble, pero puede que el sesgo aumente. 


Cada árbol se va ramificando seleccionando la variable de la muestra aleatoria que más ganancia de información nos de, también recordamos que estos árboles no se podan por lo que funcionan bien contra el ruido pero sobreajustan, lo que si podemos ajustar es la profundidad de nuestros estimadores. Seguidamente, se aplica el voto por mayoría de todos los árboles de nuestro ensemble para obtener la predicción.

In [27]:
random_forest_model = RandomForestClassifier(random_state=random_state)

forest = make_pipeline(preprocesamiento, random_forest_model)
forest

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_m249',
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=27912))])

## 3.6 Gradient Tree Boosting

`Gradient Boosting` es una técnica de aprendizaje automático que entrena modelos de manera gradual, aditiva y secuencial. A diferencia del boosting tradicional, Gradient Boosting identifica las deficiencias de los modelos débiles utilizando gradientes en la función de pérdida. En esta técnica, se emplean árboles de regresión en cada paso para predecir el error cometido, ajustando así una clasificación inicial simple, como la media o la moda. Cada iteración considera todos los modelos anteriores con igual importancia.

Esta técnica genera modelos precisos y es escalable para conjuntos de datos grandes. Sin embargo, existe el riesgo de sobreajuste, especialmente si los hiperparámetros no se configuran adecuadamente. La eficacia de Gradient Boosting radica en su capacidad para mejorar continuamente la precisión del modelo al enfocarse en las deficiencias identificadas en cada iteración.

In [28]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

gradboost = make_pipeline(preprocesamiento, gradient_boosting_model)
gradboost

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(random_state=27912))])

## 3.7 Histogram Gradient Boosting

Histogram Gradient Boosting es una optimización del algoritmo Gradient Boosting que discretiza el conjunto de datos de entrada para reducir el número de puntos de corte considerados en la construcción de los árboles de decisión. Esta técnica no requiere considerar cada valor distinto de las variables predictoras continuas como punto de corte, lo que resulta en una significativa reducción en el tiempo de entrenamiento e inferencia, mejorando la eficiencia del modelo. Además, destaca por su capacidad para manejar valores perdidos de manera implícita, sin necesidad de realizar una imputación previa.


In [29]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

histgrad = make_pipeline(preprocesamiento, hist_gradient_boosting_model)
histgrad

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function create_helmet at 0x000001EE3F5BDF70>)),
                                 ('columntransformer-1',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['t_weapon_famas',
                                                                    'ct_weapon_galilar',
                                                                    't_weapon_usps',
                                                                    'ct_weapon_glock',
                                                                    't_weapon_fiveseven',
                                                                    't_weapon_p2000',
                                                                    'ct_weap...
                                                                    'ct_weapon_mac10', ...])],
                                                    verbose_feature_names_out=False)),
                                 ('columntransformer-2',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['bomb_planted',
                                                                    'map'])],
                                                    verbose_feature_names_out=False))])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(random_state=27912))])

# 4. Evaluación de modelos

En este momento de la práctica lo que trataremos de hacer es ajustar los hiperparámetros de nuestros estimadores, esto lo haremos mediante `GridSearchCV` esta clase de scikit_learn nos permitirá hacer una búsqueda en malla de los mejores hiperparámetros, probando todas las combinaciones posibles, algo que nos puede llevar una cantidad de tiempo considerable. Debemos tener en cuenta distintas consideraciones para que el tiempo de ejecución no sea muy grande:

* El tiempo empleado en la búsqueda de hiperparámetros tiende a crecer con facilidad debido a que la cantidad de modelos a entrenar es igual al producto cartesiano de los valores de cada hiperparámetro.

* Utilizaremos una `iterated CV` que consiste en realizar $r$ iteraciones sobre nuestro $k$-fold cv. En cada iteración, se lleva a cabo una validación cruzada con $k$ folds, y luego se calcula la media de los `scores` obtenidos en todos los conjuntos de prueba. Por lo que si tenemos una `5x10-cv ` tenemos 5 repeticiones de 10 folds. En cada iteración, se realiza una nueva partición de los datos en 10 folds y se lleva a cabo el proceso de validación cruzada. Luego, se promedian los resultados de las 5 iteraciones.

* El `volumen` de nuestro conjunto de datos es bastante grande.

Lo primero que podemos ajustar, antes que reducir el volumen del conjunto de pruebas mediante una muestra , es ajustar el número de iteraciones y de `folds`. En nuestro caso probaremos con una `1x10-cv` o lo que es lo mismo una validacion cruzada de 10 folds **sin iteraciones**.

In [30]:
n_splits = 10
n_repeats = 1

cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)

Empezaremos ajustando los hiperparámetros del estimador `KNeighborsClassifier`, trateremos de conseguir el mejor modelo ajustando el número de vecinos `n_neighbors` y la forma de medir los pesos `weights` que podrán ser medidos uniformemente, donde los vecinos más alejados tienen la misma influencia que los más cercanos o con `distance` donde los más cercanos tendrán más peso a la hora de clasificar. En cuanto al número de vecinos jugaremos para ver que llega un punto donde el score converge e incluso llega a decrecer, esto ocurre porque cuanto más vecinos tengamos en cuenta más vecinos lejanos a nosotros influirán en la decisión.

-------------------------------------------
**NOTA**: Tras un problema con la etiqueta `uniform` en el hiperparámetro `weights`, no hemos podido contemplar esta etiqueta, puesto que no nos daba ningún score (Nan). Al parecer era un error en el fit() pero no hemos conseguido solucionarlo. El error al hacer el fit decia 'ndarray is not c-contigous'.

In [50]:
parametros_a_optimizar = {
    'kneighborsclassifier__n_neighbors': [3, 6, 9, 12],
    'kneighborsclassifier__weights': ["distance"],
}

k_neighbors_classifier = optimize_params(vecinos, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__n_neighbors,param_kneighborsclassifier__weights,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.400593,0.036351,2.590477,0.376622,3,distance,"{'kneighborsclassifier__n_neighbors': 3, 'knei...",0.898374,0.002409,1,0.994006,0.000179
1,0.379320,0.008304,2.248679,0.063126,6,distance,"{'kneighborsclassifier__n_neighbors': 6, 'knei...",0.895005,0.002313,2,0.993922,0.000171
2,0.392363,0.033535,2.299560,0.062765,9,distance,"{'kneighborsclassifier__n_neighbors': 9, 'knei...",0.889429,0.002949,3,0.994049,0.000162
3,0.378271,0.011665,2.249133,0.028895,12,distance,"{'kneighborsclassifier__n_neighbors': 12, 'kne...",0.888428,0.002820,4,0.994052,0.000170


Como habiamos dicho antes vemos que cuando ponemos un número de vecinos altos, nuestro score va decreciendo debido a que tiene en cuenta vecinos lejanos y no decrece mucho debido a que estamos utilizando el `weight` distance, si hubieramos utilizado el uniforme los lejanos hubieran tenido la misma influencia que los más cercanos y hubiera decrecido más.

El mejor setting para nuestro modelo `KNeighborsClassifier` es escoger un número de vecinos bajo y pesando por la distancia de estos.

-------------------------------

Ahora iremos con el `DecisionTreeClassifier` cuyos posibles parámetros a optimizar pueden ser el citerio de ramificación `criterion` que podrá ser `gini` o `entropy`, también tendremos en cuenta el hiperparámetro de máxima profundidad del árbol `max_depth` y por último `ccp_alpha` que en términos generales, introduce un término de penalización en la función de costo que se minimiza durante el proceso de construcción del árbol. Este término penaliza la complejidad del árbol, y al ajustar el valor de ccp_alpha, ajustaremos la precisión para que nuestros modelos generalicen y no tiendan a sobreajustar.

In [91]:
parametros_a_optimizar = {
    "decisiontreeclassifier__criterion": ["gini","entropy"],
    "decisiontreeclassifier__max_depth" : [10, 20, 30, None],
    "decisiontreeclassifier__ccp_alpha" : [0.0, 0.05],
}


decision_tree_classifier = optimize_params(pipeline_arbol, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_decisiontreeclassifier__ccp_alpha,param_decisiontreeclassifier__criterion,param_decisiontreeclassifier__max_depth,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
7,1.653805,0.075190,0.036527,0.007256,0.0,entropy,None,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.855302,0.003791,1,0.994368,0.000094
3,1.856747,0.232105,0.043523,0.013076,0.0,gini,None,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.853260,0.003692,2,0.994368,0.000094
2,2.196833,0.426209,0.044769,0.011346,0.0,gini,30,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.845223,0.005075,3,0.981792,0.004186
6,1.580726,0.086583,0.035545,0.006288,0.0,entropy,30,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.832581,0.004939,4,0.961438,0.004627
1,1.407996,0.026489,0.035434,0.003846,0.0,gini,20,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.803386,0.003923,5,0.909481,0.002988
5,1.633853,0.068194,0.039787,0.007368,0.0,entropy,20,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.789202,0.002530,6,0.884647,0.003400
0,0.849665,0.046415,0.033662,0.006447,0.0,gini,10,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.750388,0.003182,7,0.768885,0.001171
4,0.943483,0.080574,0.040889,0.006984,0.0,entropy,10,"{'decisiontreeclassifier__ccp_alpha': 0.0, 'de...",0.744782,0.004038,8,0.760254,0.001551
8,0.877507,0.087867,0.035181,0.006493,0.05,gini,10,"{'decisiontreeclassifier__ccp_alpha': 0.05, 'd...",0.701638,0.004900,9,0.701638,0.000544
9,1.747504,0.041094,0.039381,0.004129,0.05,gini,20,"{'decisiontreeclassifier__ccp_alpha': 0.05, 'd...",0.701638,0.004900,9,0.701638,0.000544


El mejor ajuste de hiperparámetros del `DecisionTreeClassifier` se ha obtenido dejando crecer el árbol hasta el final, es decir sin establecer ninguna profundidad y sin podar el árbol.

---------------

Para `AdaBoost` optimizaremos el número de estimadores de nuestro ensemble `n_estimators`, tmabién ajustaremos el `learning rate` el cual es un hiperparámetro que controla la contribución de cada clasificador débil a la combinación final del clasificador fuerte. Puesto que como estimador utilizaremos el `DecisionTreeClassifier` también ajustaremos sus hiperparámetros de profundidad y criterio de ramificación.


In [103]:
base_estimator = DecisionTreeClassifier(random_state=random_state)
parametros_a_optimizar = {
    "adaboostclassifier__estimator" : [base_estimator],
    "adaboostclassifier__n_estimators" : [20, 100],
    "adaboostclassifier__learning_rate" : [0.5, 1.0],
    "adaboostclassifier__base_estimator__criterion" : ['gini','entropy'],
    "adaboostclassifier__base_estimator__max_depth": [1, 3]
}


ada_boosting_classifier = optimize_params(adaptative, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_adaboostclassifier__base_estimator__criterion,param_adaboostclassifier__base_estimator__max_depth,param_adaboostclassifier__estimator,param_adaboostclassifier__learning_rate,param_adaboostclassifier__n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
15,28.101970,4.408019,0.285004,0.047765,entropy,3,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,100,{'adaboostclassifier__base_estimator__criterio...,0.766430,0.002475,1,0.780225,0.001004
13,25.521613,1.502883,0.257726,0.042105,entropy,3,"DecisionTreeClassifier(criterion='entropy', ma...",0.5,100,{'adaboostclassifier__base_estimator__criterio...,0.766134,0.003259,2,0.775644,0.001183
7,24.561380,0.291445,0.244447,0.004602,gini,3,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,100,{'adaboostclassifier__base_estimator__criterio...,0.765910,0.003410,3,0.780527,0.001248
5,26.451392,1.249143,0.265902,0.043946,gini,3,"DecisionTreeClassifier(criterion='entropy', ma...",0.5,100,{'adaboostclassifier__base_estimator__criterio...,0.765011,0.003350,4,0.775404,0.001042
12,5.047151,0.029572,0.072107,0.001001,entropy,3,"DecisionTreeClassifier(criterion='entropy', ma...",0.5,20,{'adaboostclassifier__base_estimator__criterio...,0.750878,0.004938,5,0.752631,0.001228
6,5.092536,0.053286,0.072998,0.001319,gini,3,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,20,{'adaboostclassifier__base_estimator__criterio...,0.748724,0.004387,6,0.752223,0.001925
14,6.110879,0.474778,0.093814,0.016753,entropy,3,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,20,{'adaboostclassifier__base_estimator__criterio...,0.748009,0.004420,7,0.751553,0.002412
4,6.485738,0.806733,0.099630,0.035369,gini,3,"DecisionTreeClassifier(criterion='entropy', ma...",0.5,20,{'adaboostclassifier__base_estimator__criterio...,0.747498,0.004698,8,0.749953,0.001812
3,17.301353,2.525946,0.317255,0.124682,gini,1,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,100,{'adaboostclassifier__base_estimator__criterio...,0.744874,0.004951,9,0.745568,0.000703
11,14.203830,0.050579,0.234178,0.012452,entropy,1,"DecisionTreeClassifier(criterion='entropy', ma...",1.0,100,{'adaboostclassifier__base_estimator__criterio...,0.744322,0.005473,10,0.745882,0.000824


Vemos que con `AdaBoost` lo que mejor resultado nos ha dado ha sido aumentar el número de estimadores hasta 100, una tasa de aprendizaje de 1.0, con árboles de profundidad 3 y entropia como criterio de ramificación.

---------------

Para nuestro estimador Bagging hemos tenido en cuenta que nuestro ensemble este formado bien por `DecisionTreeClassifier` o bien por el estimador `KNeighborsClassifier`, jugaremos con 5 estimadores y con 15. En los árboles ajustaremos la profundidad máxima y el criterio de ramificación, mientras que en los vecinos más cercanos como hemos visto que usando pocos vecinos obtenemos buenos resultados solamente pondremos 1 vecino.

In [98]:
base_estimator = DecisionTreeClassifier(random_state=random_state)
base_estimator_vecinos = KNeighborsClassifier(n_neighbors = 1)
parametros_a_optimizar = [
    {
    "baggingclassifier__estimator" : [base_estimator_vecinos],
    "baggingclassifier__n_estimators" : [5, 15],
    },
    {
    "baggingclassifier__estimator" : [base_estimator],
    "baggingclassifier__n_estimators" : [5, 15],
    "baggingclassifier__base_estimator__criterion" : ['gini','entropy'],
    "baggingclassifier__base_estimator__max_depth" : [25, 50],
    }
    
]

bagging_classifier = optimize_params(bagging, X_train, y_train, cv, param_grid = parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_baggingclassifier__estimator,param_baggingclassifier__n_estimators,param_baggingclassifier__base_estimator__criterion,param_baggingclassifier__base_estimator__max_depth,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
1,1.394469,0.008207,32.945939,0.322846,KNeighborsClassifier(n_neighbors=1),15,NaN,NaN,{'baggingclassifier__estimator': KNeighborsCla...,0.900182,0.002953,1,0.987734,0.000255
0,0.694337,0.016810,10.928676,0.137356,KNeighborsClassifier(n_neighbors=1),5,NaN,NaN,{'baggingclassifier__estimator': KNeighborsCla...,0.893115,0.003013,2,0.976760,0.000533
5,14.944669,0.106324,0.114406,0.004892,DecisionTreeClassifier(random_state=27912),15,gini,50,{'baggingclassifier__base_estimator__criterion...,0.884129,0.002332,3,0.991869,0.000170
9,14.862169,0.066979,0.109824,0.002348,DecisionTreeClassifier(random_state=27912),15,entropy,50,{'baggingclassifier__base_estimator__criterion...,0.883036,0.002134,4,0.991772,0.000178
3,14.332183,0.096451,0.112611,0.006210,DecisionTreeClassifier(random_state=27912),15,gini,25,{'baggingclassifier__base_estimator__criterion...,0.879217,0.002782,5,0.984769,0.000796
7,13.991222,0.270285,0.109709,0.003183,DecisionTreeClassifier(random_state=27912),15,entropy,25,{'baggingclassifier__base_estimator__criterion...,0.871222,0.003486,6,0.976973,0.001549
4,5.184243,0.031720,0.059761,0.001808,DecisionTreeClassifier(random_state=27912),5,gini,50,{'baggingclassifier__base_estimator__criterion...,0.868934,0.002087,7,0.979988,0.000415
8,5.177178,0.051768,0.063538,0.005770,DecisionTreeClassifier(random_state=27912),5,entropy,50,{'baggingclassifier__base_estimator__criterion...,0.867035,0.002211,8,0.979955,0.000565
2,5.044400,0.051990,0.061545,0.003565,DecisionTreeClassifier(random_state=27912),5,gini,25,{'baggingclassifier__base_estimator__criterion...,0.860346,0.001740,9,0.968858,0.001050
6,4.856218,0.035942,0.063034,0.004756,DecisionTreeClassifier(random_state=27912),5,entropy,25,{'baggingclassifier__base_estimator__criterion...,0.850666,0.003934,10,0.956772,0.003268


Vemos que la mejor configuración la hemos obtenido haciendo un ensemble de `KNeighborsClassifier` con solamente un vecino, pero observamos que cuantos más estimadores usemos en nuestro ensemble mejoraremos el resultado.

En el caso de usar árboles como estimadores también vemos que cuanto más lo dejemos crecer mejor actuará.

-----------------

Para `RandomForest` consideraremos como parámetros a ajustar el número de estimadores del ensemble, el criterio de ramificación y el número de características que tendremos en cuenta para ramificar cada nodo de un árbol. Como aclaración a esto último podemos aportar lo siguiente: 

* Si nuestro `max_features` = `sqrt` únicamente tendremos en cuenta $\sqrt X$ variables predictoras en cada nodo, escogidas aleatoriamente, siendo X el número total de variables predictoras de nuestro *dataset*.

In [33]:
parametros_a_optimizar = {
    "randomforestclassifier__n_estimators" : [20, 100],
    "randomforestclassifier__criterion": ['gini', 'entropy'],
    "randomforestclassifier__max_features": ['sqrt', 'log2']
}

random_forest_classifier = optimize_params(forest, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__criterion,param_randomforestclassifier__max_features,param_randomforestclassifier__n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
7,16.440415,0.307688,0.385073,0.013073,entropy,log2,100,{'randomforestclassifier__criterion': 'entropy...,0.904787,0.002090,1,0.994367,0.000094
3,17.721791,1.016711,0.439474,0.034386,gini,log2,100,"{'randomforestclassifier__criterion': 'gini', ...",0.904001,0.001123,2,0.994361,0.000092
5,20.926644,1.250062,0.405980,0.056734,entropy,sqrt,100,{'randomforestclassifier__criterion': 'entropy...,0.902898,0.001696,3,0.994363,0.000091
1,19.455089,0.634983,0.396219,0.033705,gini,sqrt,100,"{'randomforestclassifier__criterion': 'gini', ...",0.902500,0.001767,4,0.994364,0.000091
6,3.804338,0.288577,0.110332,0.011523,entropy,log2,20,{'randomforestclassifier__criterion': 'entropy...,0.897751,0.002147,5,0.993272,0.000146
2,3.633278,0.312303,0.115798,0.017934,gini,log2,20,"{'randomforestclassifier__criterion': 'gini', ...",0.897619,0.001885,6,0.993273,0.000172
4,4.365900,0.273983,0.112054,0.013866,entropy,sqrt,20,{'randomforestclassifier__criterion': 'entropy...,0.896107,0.002697,7,0.993155,0.000150
0,3.951902,0.205202,0.108813,0.013635,gini,sqrt,20,"{'randomforestclassifier__criterion': 'gini', ...",0.895168,0.002727,8,0.993164,0.000161


Vemos que la mejor configuración para nuestro Random Forest es utilizar 100 estimadores, como criterio utilizar entropia y la función para escoger el número de características en cada nodo será el logaritmo en base 2 (log2).

------------------

Ahora vamos con el estimador `Gradient Boosting` del que trataremos de optimizar los hiperparámetros de `learning rate` que controla la contribución de cada árbol, cuanto menor sea este valor, más estimadores necesitaremos para que el modelo ajuste. También ajustaremos otros como el criterio para medir la calidad de cada split mediante el error cuadrático o el error cuadrático medio de friedman que introduce una mejora al tradicional, por último ajustaremos la profundidad máxima de cada árbol.

In [99]:

parametros_a_optimizar = {
    "gradientboostingclassifier__learning_rate" : [0.01, 0.1],
    "gradientboostingclassifier__n_estimators" : [20, 50],
    "gradientboostingclassifier__criterion" : ['friedman_mse','squared_error'],
    "gradientboostingclassifier__max_depth" : [1, 3],
}
gradient_boosting_classifier = optimize_params(gradboost, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gradientboostingclassifier__criterion,param_gradientboostingclassifier__learning_rate,param_gradientboostingclassifier__max_depth,param_gradientboostingclassifier__n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
7,10.493635,1.999165,0.046661,0.004008,friedman_mse,0.1,3,50,{'gradientboostingclassifier__criterion': 'fri...,0.746497,0.006088,1,0.747169,0.001102
15,8.957469,0.084996,0.043483,0.001558,squared_error,0.1,3,50,{'gradientboostingclassifier__criterion': 'squ...,0.746497,0.006088,1,0.747169,0.001102
6,4.004419,0.276734,0.042391,0.004173,friedman_mse,0.1,3,20,{'gradientboostingclassifier__criterion': 'fri...,0.733835,0.006824,3,0.734551,0.001044
14,4.864092,0.957786,0.048776,0.017685,squared_error,0.1,3,20,{'gradientboostingclassifier__criterion': 'squ...,0.733835,0.006824,3,0.734551,0.001044
5,3.658524,0.293044,0.038098,0.001161,friedman_mse,0.1,1,50,{'gradientboostingclassifier__criterion': 'fri...,0.715260,0.004514,5,0.715885,0.001916
13,3.831252,0.580856,0.042487,0.006757,squared_error,0.1,1,50,{'gradientboostingclassifier__criterion': 'squ...,0.715260,0.004514,5,0.715885,0.001916
3,9.642528,0.514787,0.048661,0.007922,friedman_mse,0.01,3,50,{'gradientboostingclassifier__criterion': 'fri...,0.710042,0.006758,7,0.710109,0.002028
11,11.980436,1.943740,0.057645,0.018979,squared_error,0.01,3,50,{'gradientboostingclassifier__criterion': 'squ...,0.710042,0.006758,7,0.710109,0.002028
4,1.616375,0.097345,0.038006,0.002201,friedman_mse,0.1,1,20,{'gradientboostingclassifier__criterion': 'fri...,0.706213,0.004527,9,0.706230,0.000510
12,1.512753,0.020198,0.036698,0.000607,squared_error,0.1,1,20,{'gradientboostingclassifier__criterion': 'squ...,0.706213,0.004527,9,0.706230,0.000510


Con la tabla de resultados podemos observar que utilizando un número medio de estimadores, una profundidad baja y un LR de 0.1 obtenemos el mejor resultado. También podemos observar que el criterio para medir la calidad en cada nodo no es determinante en este modelo.

------------------------

Por último vamos con nuestro `Histogram Gradient Boosting` del que ajustaremos la tasa de aprendizaje que tiene un papel similar al Gradient Boosting, con el hiperparámetro max_iter lo que estaremos ajustando es el número de iteraciones de boosting (por cada iteración se añade un estimador al residual), con otras palabras el número de estimadores del ensemble. En otros casos hemos controlado la profundidad, pero aquí haremos algo parecido que será controlar la extensión del árbol limitando el número de hojas de cada estimador con el hiperparámetro `max_leaf_nodes`.

In [45]:

parametros_a_optimizar = {
    "histgradientboostingclassifier__learning_rate" : [0.01, 0.05],
    "histgradientboostingclassifier__max_iter" : [20, 50, 100],
    "histgradientboostingclassifier__max_leaf_nodes" : [127, 3000],
}

hist_gradient_boosting_classifier = optimize_params(histgrad, X_train, y_train, cv, **parametros_a_optimizar)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_histgradientboostingclassifier__learning_rate,param_histgradientboostingclassifier__max_iter,param_histgradientboostingclassifier__max_leaf_nodes,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
11,68.834987,0.686220,0.197910,0.011825,0.05,100,3000,{'histgradientboostingclassifier__learning_rat...,0.885263,0.001642,1,0.979103,0.000286
9,32.797152,0.366832,0.114260,0.008020,0.05,50,3000,{'histgradientboostingclassifier__learning_rat...,0.864043,0.002470,2,0.946044,0.000506
5,59.816389,2.050131,0.154262,0.012426,0.01,100,3000,{'histgradientboostingclassifier__learning_rat...,0.839893,0.001964,3,0.906906,0.001157
7,12.808135,0.116488,0.070525,0.008078,0.05,20,3000,{'histgradientboostingclassifier__learning_rat...,0.838116,0.002347,4,0.905455,0.000951
3,30.657508,1.167104,0.090295,0.007824,0.01,50,3000,{'histgradientboostingclassifier__learning_rat...,0.821869,0.002854,5,0.882565,0.001981
1,12.213279,0.447729,0.065975,0.013759,0.01,20,3000,{'histgradientboostingclassifier__learning_rat...,0.804397,0.002678,6,0.857808,0.002950
10,5.865552,0.061249,0.100693,0.013526,0.05,100,127,{'histgradientboostingclassifier__learning_rat...,0.802743,0.003784,7,0.832926,0.001381
8,3.246644,0.054319,0.071081,0.010102,0.05,50,127,{'histgradientboostingclassifier__learning_rat...,0.780951,0.004344,8,0.799796,0.001135
4,10.533990,11.423951,0.095413,0.013617,0.01,100,127,{'histgradientboostingclassifier__learning_rat...,0.767053,0.004611,9,0.778358,0.000572
6,1.623514,0.017934,0.048819,0.006111,0.05,20,127,{'histgradientboostingclassifier__learning_rat...,0.766645,0.005243,10,0.777913,0.000840


Como vemos el mejor resultado se ha obtenido escogiendo como número máximo de hojas 3000, este ha sido el hiperparámetro más significativo, seguidamente vemos que cuantas más iteraciones y por tanto más estimadores el *acuraccy* también aumenta y por último escoger un learning rate de 0.05, esa sería la mejor configuración para este ensemble.

# 5. Construcción y validación del modelo final

Tras evaluar todos nuestros modelos eligiendo los mejores hiperparámetros, llega el momento de elegir el modelo que le entregaremos a nuestro cliente y una medida de rendimiento.


Como modelo le entregaremos el que mejor resultado nos haya dado, este resultado lo podemos obtener mediante las métricas de rendimiento como `accuracy`, `recall`, `f1-score`... en nuestro caso utilizaremos estos tres. Pero al tener un 10-cv tenemos distintos valores

* **¿Deberiamos devolver la mejor? La respuesta es que no porque estariamos siendo demasiado optimistas si le decimos que nuestro modelo acierta el 90% de las veces, cuando la media es de un 80%**. La respuesta correcta sería que al cliente le daremos la media de la 10-cv del *accuracy* del mejor modelo.

En cuanto al modelo...
* **¿Qúe modelo le dariamos a nuestros clientes de nuestra 10-cv? La respuesta es que ninguno, puesto que esos modelos han sido entrenados con una parte del train pero no con todo el train**. Por lo que en nuestra validación cruzada cogeremos el modelo con mejor media de *acurracy* y lo entrenaremos con todos los datos de entrenamiento, este será el modelo que le entregaremos al cliente.



In [166]:
estimators = [k_neighbors_classifier, decision_tree_classifier, ada_boosting_classifier, bagging_classifier, random_forest_classifier, gradient_boosting_classifier, hist_gradient_boosting_classifier]
 
metrics = ["accuracy", "f1", "recall"]

evaluate_estimators(estimators, metrics, X_test, y_test)

,accuracy,f1,recall,Tiempo de inferencia
KNeighborsClassifier,0.907279,0.909156,0.910103,5.966047
DecisionTreeClassifier,0.862184,0.864715,0.863953,0.052828
AdaBoostClassifier,0.763826,0.763459,0.747616,0.622299
BaggingClassifier,0.908096,0.909942,0.910744,92.242324
RandomForestClassifier,0.910097,0.911567,0.908902,1.054183
GradientBoostingClassifier,0.745241,0.734562,0.691451,0.068823
HistGradientBoostingClassifier,0.892452,0.894321,0.892637,0.404881


**Tiempo de ejecución GridSearch** :
* `KNeighborsClassifier` : 15 minutos y 48 segundos
* `DecisionTreeClassifier` : 4 minutos y 46 segundos
* `AdaBoostClassifier` : 38 minutos y 57 segundos
* `BaggingClassifier` : 85 minutos y 59 segundos
* `RandomForestClassifier` : 18 minutos y 45 segundos
* `GradientBoostingClassifier` : 14 minutos y 53 segundos
* `HistGradientBoostingClassifier` : 43 minutos y 43 segundos



# 6. Conclusiones

Como hemos observado en nuestros resultados en algunos modelos el tiempo de entrenamiento es muy alto y el tiempo de inferencia es bajo, esto nos ocurre en los árboles de Decisión y en los ensembles que usan este como estimador. Sin embargo tenemos modelos cuyo tiempo de entrenamiento es bajo y el tiempo de inferencia es el que ocupa la mayor parte del tiempo de la búsqueda en malla y más tarde en el test, es el caso del algoritmo de los vecinos más cercanos, este último actúa bajo el paradigma ***Lazy Learning***.

Para elegir nuestro mejor modelo seguiremos un poco el principio de la **Navaja de Ockham**, si tenemos un modelo que nos da práctiacamente los mismos resultados que otro, siempre nos quedaremos con el más sencillo, entendiendo por sencillo el tiempo que nos ha llevado hacer la búsqueda en malla + el tiempo de inferencia. Vemos que para obtener un buen resultado con `AdaBoost` hemos necesitado un tiempo muy elevado durante la búsqueda en malla, por lo que este estaría descartado como nuestro mejor modelo. Por otra parte tenemos nuestro `DecisionTreeClassifier` que nos ha llevado un tiempo muy reducido comparado con los demás modelos, y con un *acurraccy* que no es el mejor pero es bastante bueno. En cuanto a los ensembles detacaremos el `RandomForest` cuyo tiempo en el GridSearch es un tiempo bueno teniendo en cuenta que es un ensemble y el tiempo de inferencia es bajo y si a eso le sumamos que nos da el mejor score tanto en la media de la validación cruzada como cuando le pasamos el test al modelo entrenado con todos los datos del train, tenemos un modelo excelente. Si analizamos el `KNeighborsClassifier` podemos observar que al ser un método *Instanced-based* todo el tiempo lo dedica a validar en la validación cruzada y a hacer la inferencia del test, obteniendo un *acuraccy* casi a la altura del `RandomForest` y utilzando pocos vecinos, por lo que la sencillez de este hace que esté por encima incluso del `RandomForest`.

En cuanto a los ensembles hay dos que nos dan malos resultados comparados con los demás, estos son `AdaBoost` y `GradientBoosting` una de las razones puede ser la cantidad de estimadores que hemos utilizado, puesto que estos dos algoritmos trabajan con *weak learners* les puede ser más dificil generalizar y por tanto con un número más elevado de estimadores podriamos haber mejorado los resultados. Pero si hubiera un motivo principal, prodriamos decir que no todos los clasificadores y ensembles funcionan bien en todos los problemas es por eso que en este tipo de prácticas es esencial utilizar varios tipos y quedarnos con el que mejor resultados nos de.